In [1]:
from transformers import AutoTokenizer,AutoModelForCausalLM
import os
import torch

device = 'cuda' if torch.cuda.is_available() else 'mps' if torch.backends.mps.is_available() else 'cpu'

model_path = os.path.expanduser('~/models/Qwen/Qwen3-0.6B')

model = AutoModelForCausalLM.from_pretrained(model_path,device_map=device,torch_dtype=torch.float16)
tokenizer = AutoTokenizer.from_pretrained(model_path)


/Users/daijunjie/miniconda3/envs/langchain-env/lib/python3.12/site-packages/requests/__init__.py:86: RequestsDependencyWarning: Unable to find acceptable character detection dependency (chardet or charset_normalizer).
  warnings.warn(
/Users/daijunjie/miniconda3/envs/langchain-env/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
print(tokenizer)

Qwen2TokenizerFast(name_or_path='/Users/daijunjie/models/Qwen/Qwen3-0.6B', vocab_size=151643, model_max_length=131072, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'eos_token': '<|im_end|>', 'pad_token': '<|endoftext|>', 'additional_special_tokens': ['<|im_start|>', '<|im_end|>', '<|object_ref_start|>', '<|object_ref_end|>', '<|box_start|>', '<|box_end|>', '<|quad_start|>', '<|quad_end|>', '<|vision_start|>', '<|vision_end|>', '<|vision_pad|>', '<|image_pad|>', '<|video_pad|>']}, clean_up_tokenization_spaces=False, added_tokens_decoder={
	151643: AddedToken("<|endoftext|>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	151644: AddedToken("<|im_start|>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	151645: AddedToken("<|im_end|>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	151646: AddedToken("<|object_ref_start|>", rstrip=False, lstrip=False, singl

# 1.Tokenizer 

In [14]:
from torch import return_types


text = '你是一个小模型!'
# encoded = tokenizer.encode(text)
encoded = tokenizer([text],return_tensors='pt').to('mps')
print(f'encoded: {encoded}')

print(f'decoded: {tokenizer.decode(encoded['input_ids'][0])}')

encoded: {'input_ids': tensor([[ 56568, 101909,  30709, 104949,      0]], device='mps:0'), 'attention_mask': tensor([[1, 1, 1, 1, 1]], device='mps:0')}
decoded: 你是一个小模型!


## 把tokenzier 编码结果作为模型的输入

In [ ]:
generated_ids = model.generate(**encoded,max_length = 200)
# generated_ids

/Users/daijunjie/miniconda3/envs/langchain-env/lib/python3.12/site-packages/transformers/pytorch_utils.py:335: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  test_elements = torch.tensor(test_elements)


tensor([[ 56568, 101909,  30709, 104949,      0,  49434,    239,  99601,  30534,
          61443,  46944, 101888, 104455,  18493,  99328, 101047,  99892,  33108,
          99564,   9370, 104734,   3837,  85106,  61443,  46944, 104734, 106367,
           3837, 106367,  30534, 102298,   2073, 104455,    854,  33108,   2073,
          99328,    854, 100369, 105291,   1773, 106367,  30534, 101137, 104380,
         104734, 104272,   3837, 106367,  30534, 110485,   5373,  18830, 104380,
          33071,   5373,  18830,  99556, 100162,   1773, 106367,  85106, 102298,
           2073,  99328,    854,  33108,   2073, 104455,    854, 100369, 105291,
           3837,  91572, 106367,  30534, 101137, 104380, 104734, 104272,   3837,
         106367,  30534, 110485,   5373,  18830, 104380,  33071,   5373,  18830,
          99556, 100162,   1773, 106367,  85106, 102298,   2073,  99328,    854,
          33108,   2073, 104455,    854, 100369, 105291,   3837,  91572, 106367,
          30534, 101137, 104

# 用tokenzier解码模型的输出

In [ ]:
# tokenizer.decode(generated_ids[0])
tokenizer.batch_decode(generated_ids)[0]

'你是一个小模型! 我现在要写一个关于人工智能在社会中的应用和影响的论文，需要写一个论文题目，题目要包含“人工智能”和“社会”两个关键词。题目要符合学术论文的要求，题目要简洁、有学术性、有研究价值。题目需要包含“社会”和“人工智能”两个关键词，同时题目要符合学术论文的要求，题目要简洁、有学术性、有研究价值。题目需要包含“社会”和“人工智能”两个关键词，同时题目要符合学术论文的要求，题目要简洁、有学术性、有研究价值。题目需要包含“社会”和“人工智能”两个关键词，同时题目要符合学术论文的要求，题目要简洁、有学术性、有研究价值。题目需要包含“社会”和“人工智能”两个关键词，同时题目要符合学术论文的要求，题目要简洁、有学术性、有研究价值。\n好的，现在我需要帮用户写'

# 2.模型推理

In [ ]:
user_text = "黄仙洞是什么?"
messages = [{'role': 'user', 'content': user_text}]
user_formatted_text = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True,
)
user_formatted_text

'<|im_start|>user\n黄仙洞是什么?<|im_end|>\n<|im_start|>assistant\n'

In [11]:
model_inputs = tokenizer(user_formatted_text,return_tensors='pt').to('mps')
model_inputs

{'input_ids': tensor([[151644,    872,    198,  99789, 100717, 100743, 102021,     30, 151645,
            198, 151644,  77091,    198]], device='mps:0'), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]], device='mps:0')}

In [12]:
# 3. 生成回答
with torch.no_grad():
    generated_ids = model.generate(
        **model_inputs,
        max_new_tokens=512,
        temperature=0.7,
        do_sample=True,
        pad_token_id=tokenizer.eos_token_id,
        eos_token_id=tokenizer.eos_token_id,
    )

# 4. 解码回答（只取新生成的部分）
generated_tokens = generated_ids[0][len(model_inputs['input_ids'][0]):]
response = tokenizer.decode(generated_tokens, skip_special_tokens=True)

/Users/daijunjie/miniconda3/envs/langchain-env/lib/python3.12/site-packages/transformers/pytorch_utils.py:335: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  test_elements = torch.tensor(test_elements)


In [13]:
print(response)

<think>
好的，用户问“黄仙洞是什么？”。首先，我需要确认用户指的是哪个黄仙洞。因为可能存在多个同名的地点，比如中国的一个地方，或者国外的景点。首先，我应该先明确用户所说的“黄仙洞”具体指的是哪个地方。如果用户没有提供更多信息，可能需要进一步询问。

接下来，我需要考虑用户的潜在需求。用户可能对黄仙洞感兴趣，想知道它的位置、特色或者有什么特别之处。可能用户在旅行或者计划游览时遇到了这个名称，或者有其他目的。因此，我需要提供准确的信息，同时保持回答的友好和帮助性。

另外，用户可能没有意识到自己可能指的是多个地方，所以需要明确回答，避免用户混淆。同时，考虑到可能的文化或地理差异，需要确保信息的准确性和相关性。

最后，检查回答是否符合中文表达习惯，信息是否完整，是否需要进一步帮助。确保回答简洁明了，同时涵盖主要方面，比如地理位置、特色、历史或文化元素，以满足用户的需求。
</think>

黄仙洞是位于中国湖南省张家界市的一个著名自然景区。它以独特的喀斯特地貌闻名，是世界自然遗产地之一。洞内有“天洞”、“仙洞”等名称，洞内有洞穴、瀑布、溶洞等多种自然景观，环境幽美，被誉为“中国喀斯特地貌的典范”。黄仙洞不仅是中国著名的旅游景点，也是研究喀斯特地质学的重要地点。


## 模型推理封装为函数


In [14]:
def chat_with_model(user_input, model, tokenizer):
    """完整的聊天推理函数"""
    # 1. 格式化输入
    messages = [{'role': 'user', 'content': user_input}]
    formatted_text = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True
    )
    
    # 2. 分词并转换为tensor
    model_inputs = tokenizer(formatted_text, return_tensors='pt').to('mps')
    
    # 3. 生成回答
    with torch.no_grad():
        generated_ids = model.generate(
            **model_inputs,
            max_new_tokens=512,
            temperature=0.7,
            do_sample=True,
            pad_token_id=tokenizer.eos_token_id,
            eos_token_id=tokenizer.eos_token_id,
        )
    
    # 4. 解码回答（只取新生成的部分）
    generated_tokens = generated_ids[0][len(model_inputs['input_ids'][0]):]
    response = tokenizer.decode(generated_tokens, skip_special_tokens=True)
    
    return response

# 使用示例
user_question = "介绍下湖北荆门黄仙洞"
answer = chat_with_model(user_question, model, tokenizer)
print(f"问题: {user_question}")
print(f"回答: {answer}")

/Users/daijunjie/miniconda3/envs/langchain-env/lib/python3.12/site-packages/transformers/pytorch_utils.py:335: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  test_elements = torch.tensor(test_elements)


问题: 介绍下湖北荆门黄仙洞
回答: <think>
好的，用户想了解湖北荆门黄仙洞。首先，我需要确认这个洞穴的位置和基本信息。荆门位于湖北省，黄仙洞应该是一个特定的自然景观。

接下来，我需要考虑用户的需求。他们可能对地质、历史、文化或者旅游感兴趣。用户可能想知道洞穴的形成过程，或者有什么特别的活动。需要确保信息准确，避免错误。

然后，我要检查是否有相关的资料或可靠的信息来源。可能需要查阅地质资料或旅游指南，确保信息的准确性。同时，要使用中文进行介绍，保持回答的流畅和自然。

另外，要注意用户可能的深层需求，比如是否对自然景观感兴趣，或者有旅游计划。回答时要突出洞穴的独特之处，比如地质特征或文化意义，以吸引他们进一步探索。

最后，确保回答结构清晰，分点列出关键信息，方便用户快速获取。同时，保持口语化，避免过于正式，让用户感觉亲切自然。
</think>

湖北荆门黄仙洞位于荆门市，是湖北省著名的地质和自然景观之一。以下是关于黄仙洞的基本介绍：

1. **地理位置**：黄仙洞位于荆门市东北部，地处长江中游的黄龙山东麓，是长江水系的重要组成部分。

2. **地质特征**：洞穴形成于侏罗纪时期，由石灰岩层构成，具有独特的地质结构和水系景观，是典型的溶洞地貌。

3. **历史意义**：黄仙洞曾是古代祭祀活动的场所，是荆门地区重要的文化遗迹，现为荆门市重点文物保护单位。

4. **旅游特色**：洞内设有观景台和生态观景台，游客可俯瞰洞穴全景，感受自然的壮丽景色。洞穴内部还保存有丰富的化石和植物标本。

5. **文化内涵**：黄仙洞不仅具有自然景观，还承载着荆门的历史文化，是研究地质和人类文明的重要资源。

黄仙洞以其独特的地质构造和丰富的自然景观，吸引了众多游客前来探秘，是湖北省重要的自然与文化旅游目的地之一。
